# Day 2 Exercises (NumPy + Matplotlib)

## Part 1: Basic NumPy Operations
a) Generate an array of numbers 0-24. Reshape to a 5x5 matrix.

b) Extract the diagonal of this matrix.

c) Multiply the matrix by an identity matrix of the same shape. Confirm that it is identical to the original.

Hint: Use `np.all` command to confirm all equal. 

d) Join the matrix with itself and return a new matrix with shape (2,5,5).

e) Compute the mean of the concatenated matrix along the first axis. Confirm its equal to the original matrix.

f) Return the indices of the matrix where the elements are greater than 15.

g) Using `np.where`, set all elements of the matrix greater than 15 to 1, else 0.


h) Set all elements of the matrix greater than 15 to 2, less than 5 to 1, else 0.

Hint: `np.where` can be passed as an input to `np.where`.

i) Return the lower triangle of the original matrix.

j) Define a demean function.

k) Apply the demean function across each row of the matrix.

l) Implement vector projection.

We'll do this by replicating the figure at the top of Page 6 of our linear algebra intro slides. That means we'll make the projection of $v$ onto $u$, as well as the residual vector. Can you verify that the residual vector is orthogonal to the unit vector $u$?

In [ ]:
def plot_vector(vector, start=np.array([0, 0]), color='k', **kwargs):
    # This little function is a light wrapper over a matplotlib
    # function for drawing arrows. See docs for more options.
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.arrow.html#matplotlib.pyplot.arrow
    plt.arrow(
        start[0], start[1], vector[0], vector[1],
        head_width=.05, color=color, length_includes_head=True,
        **kwargs)

# This is handy since it makes right angles easier to see.
plt.axis('equal')

u = np.array([1, .3])
u = u / norm(u)
plot_vector(u, color='red', label='projected onto')

v = np.array([1, 1.2])
plot_vector(v, color='green', label='original')

## YOUR SOLUTION HERE:
projected = 
residual = 

plot_vector(projected, color='blue', label='projected')
plot_vector(residual, start=projected, color='purple', label='residual')

plt.legend()

m) Can you find a vector that's orthogonal to $[1.25, 1.75]$?

In [ ]:
v = np.array([1.25, 1.75])

# YOUT SOLUTION HERE:
orthogonal_vector =

assert np.isclose(v@orthogonal_vector, 0)

## Part 2: Spike Detection

In the following exercises, you will be manipulating, analyzing, and visualizing preprocessed extracellular electrophysiological data. Specifically, you will be working with a 10s recording taken from a [nerve in the abdomen of a crayfish](https://www.science.smith.edu/departments/neurosci/courses/bio330/labs/L7pix/hookelectr.jpg). Action potentials are visually apparent throughout the entire recording. 

This exercise will focus primarily on the problem of **spike detection**. Before diving in, we recommend you take a few moments to skim [Rey et al. (2015)](https://www.sciencedirect.com/science/article/pii/S0361923015000684), which will help frame the problem and describe the steps involved. Specifically, we recommend reading up to and including Section 2.2, *Spike Detection*.

First, we load the data.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load data.
npz = np.load('spikes.npz')   # spikes.npz is a compressed NumPy object containing the data.
data = npz['data'] * 1e6      # data is the recording itself, which we convert to microvolts.
times = npz['times']          # times contains the timestamp for each sample of recording.

a) First, check the metadata of *data* and *times*. How many samples (elements) are in *data*? How long (in seconds) is the recording? 

b) Plot the entire raw recording with time (in seconds) on the x-axis and voltage (in microvolts) on the y-axis. 

In this simple circuit, different types of spikes can be identified according to their amplitudes. Do multiple types of spikes appear to be present?

c) Try "zooming in" on different sections of recording. That is, try plotting smaller segments (0.5s, 1.0s) of recordings. How would you restrict your graph to a smaller slice of data? (Hint: this can be done without changing the inputs to `ax.plot`)

d) [Rey et al. (2015)](https://www.sciencedirect.com/science/article/pii/S0361923015000684) suggest a simple spike detection technique via data-driven amplitude thresholding. Specifically, they propose using an amplitude threshold defined as multiple of an estimate of the standard deviation of the noise:

$$ \text{threshold} = k \cdot \hat{\sigma}_n $$

where $k$ is a constant typically between 3-5; and $\hat{\sigma}_n$ is an estimate of the standard deviation of the noise, defined as:

$$ \hat{\sigma}_n = \frac{\text{median} \left( |X| \right)}{0.6745} $$ 

where $|X|$ is the absolute value of the raw data.

Write a function that returns the amplitude threshold as defined above. The function should accept as arguments the raw data, $X$, and the constant, $k$. 

e) Using your function, compute different amplitude thresholds for different values of $k$. Overlay each threshold as a horizonal line on a plot of the raw recording. (Hint: google a horizontal line function in Matplotlib.)

For different threshold levels (values of $k$), how sensitive does the threshold seem to be? In other words, how good is your threshold at detecting "true" spikes from "noise"? (To assess this, it may be helpful to zoom in on specific segments of the data.)

f) Next we need a function that can detect segments of the raw signal that exceed the threshold. This ultimately becomes a clustering problem (i.e. identifying "islands" of signal rising above an "ocean of noise"). 

Though this is definitely doable with core NumPy, the SciPy library has built-in functions specifically written for these purposes. Because these functions are beyond the scope of the bootcamp, we have provided a peak finding function for you. The function relies on the `measurements` tools from scipy.ndimage. For a tutorial, see [here](https://dragly.org/2013/03/25/working-with-percolation-clusters-in-python/).

The function, `peak_finder`, accepts a raw data trace and a threshold. It then finds all clusters of samples above the given threshold and returns:
- the indices (positions in the array) corresponding to the peak of each cluster
- the magnitude of the signal at each peak

In [ ]:
def peak_finder(X, thresh):
    """Simple peak finding algorithm.
    
    Parameters
    ----------
    X : array_like, shape (n_times,)
        Raw data trace.
    thresh : float
        Amplitude threshold.
        
    Returns
    -------
    peak_loc : array_like, shape (n_clusters,)
        Index of peak amplitudes.
    peak_mag : array_like, shape (n_clusters,)
        Magnitude of peak amplitudes.
    """
    import numpy as np
    from scipy.ndimage import measurements
    
    ## Error-catching.
    assert X.ndim == 1
    
    ## Identify clusters.
    clusters, ix = measurements.label(X > thresh)
    
    ## Identify index of peak amplitudes. 
    peak_loc = np.concatenate(measurements.maximum_position(X, labels=clusters, index=np.arange(ix)+1))
    
    ## Identify magnitude of peak amplitudes.
    peak_mag = measurements.maximum(X, labels=clusters, index=np.arange(ix)+1)
    return peak_loc, peak_mag

g) Apply the peak detection algorithm to the raw data using a threshold of your choice (recommendation: $k=6$). How many spikes are detected? If you adjust the threshold up or down, how does this change the number of spikes detected?

h) Now for a challenge. First, plot the first second of the recording. Next, using a scatterplot (or any other method you can think of), plot the peak for each detected spike. In other words, each point in the scatter plot should be (time of spike (s), amplitude of spike (uV)). 

(Hint: *peak_loc* gives you the index of each spike in the data array. Remember that *data* and *times* have the same number of elements.)

i) Remake the plot above using different thresholds. For different threshold levels (values of $k$), how sensitive does the threshold seem to be? In other words, how good is your threshold at detecting "true" spikes from "noise"? (To assess this, it may be helpful to zoom in on different segments of the data.)

j) Based on your exploration above, choose a threshold value. You will use this threshold value for the remainder of this exercise. 

Once you've selected your threshold, use `peak_finder` again to detect spikes in the recording. Then plot a histogram of the spike amplitudes it detects (try bins of 0-150 in increments of 5 uV). 

How many spikes are detected? Based on their amplitudes, how many types of spikes do there appear to be?

k) Based on the plot above, define a set of boundaries that divides the spikes into different clusters. Find a NumPy function that assigns an each spike amplitude to a category based on your defined boundaries. (Hint: there are at least three.) Store the category labels in a new variable, *labels*. 

After binning, how many spikes are in each cluster?

m) **CHALLENGE**: Action potentials last roughly 1-2 milliseconds. With this in mind, extract a 3 ms window around each detected spike; that is, extract 1.5 ms of samples on either side of the detected peak. Store each epoch in a separate list according to its cluster. (Hint: The data were recorded at 10 KHz meaning there are 10 samples per millisecond.)

This is a challenging exercise that can be solved in many different ways. Before jumping in, take a few minutes to brainstorm pseudocode with your team. That is, write out in words each step you need to take to solve this problem.

n) Compute the average spike waveform for each cluster. Then, plot each average waveform in a single plot. Add a legend denoting the spike cluster.

# Bonus Exercises (Pandas + Seaborn)

### Background: Deep Brain stimulation

In this set of exercises, you will perfrom part of the analysis from [Widge et al. (2019)](https://www.nature.com/articles/s41467-019-09557-4), *Deep brain stimulation of the internal capsule enhances human cognitive control and prefrontal cortex function*. In this paper, the authors found that deep brain stimulation (DBS) of the ventral striatum improved performance on a task of cognitive flexibility in a sample of 14 patients with severe depression. Specifically, response times on the task decreased (speeded up) with their DBS devices turned on.

To recreate some of the analyses and figures, you will use Pandas, Seaborn, and SciPy.stats. 

### Methods
To probe cognitive flexibility, the authors employed a modified version of the Multi-Source Interference Task (MSIT). The MSIT requires subjects to identify which of a set of three numbers is different than its neighbors. Subjects must keep three fingers of their right hand positioned over response keys corresponding to the digits 1-3. In **Control** (non-interference) trials, the target is in the same spatial position as its corresponding response key, and the flanking digits are not valid responses (i.e., they are 0s). In **Interference trials**, the target is out-of-position relative to its corresponding key-press and is flanked by other viable targets. 

Each block of trials contained 72 Control and 72 Interference trials. To prevent response sets or habituation, trial sequence in each block was pseudo-randomized so that subjects never had more than two trials in a row that shared the same interference level or desired response finger. This highly interleaved trial design was expected to place greater demands on cognitive control systems by reducing predictability of the stimuli. 

Patients performed this task twice: first with their DBS device turned **ON**, and then with their DBS device turned **OFF**.

### Data

The file *dbs.csv* contains the raw behavioral data from the 14 patients. The data is organized into six columns:

- *Subject:* the unique subject identifier.
- *Trial:* the trial number per block.
- *DBS:* the status of the DBS device (ON = 1, OFF = 0)
- *Interference:* the type of trial (Control = 0, Interference = 1)
- *Accuracy:* the accuracy of the response of the trial (Correct = 1, Incorrect = 0)
- *RT:* the response time on that trial (in seconds).

## Section 1: Preprocessing

a) Read in the DataFrame from the CSV file, *dbs.csv*.

b) Using `DataFrame.value_counts`, confirm there are 14 patients with 288 trials each.

c) Using `DataFrame.groupby`, compute the average accuracy per participant. Should any participant be excluded (e.g. lower than 70% accuracy)?

d) Response times for trials with incorrect responses are typically biased. In other words, they tend to be systematically faster than the average response (i.e. fast error) or slower than the average response (i.e. slow errors). 

Remove all rows corresponding to trials with incorrect responses. Confirm no incorrect responses remain.

e) Similarly, participants occasionally make unrealistic fast responses (e.g. < 300 ms). These typically correspond to accidental button presses or slips of the finger.

Remove all rows corresponding to trials with response times faster than 300 ms. Confirm the mininum response time per participant is above this threshold.

f) Reaction times are typically right-skewed. That is, on average, a distribution of reaction times is asymmetric with more responses falling on the slower side.

Using `sns.FacetGrid` (or any other plotting method of your choosing), plot the RT distribution per participant. Confirm that all (or most) RT distributions are right-skewed.

g) To correct for right-skew, some analysts apply a log transform to reaction time data. 

Make a new column in the DataFrame, **logRT**, that is applying a log transform the RT data.

h) Write a z-score function. The z-score is defined as:

$$ Z = \frac{X - \text{mean}(X)}{\text{sd}(X)} $$

i) Apply the z-score transformation to the log reaction times (logRT) *per subject*. This has the effect of normalizing all participants' data to be in a similar range (e.g. removes baseline differences in RTs). Store the z-scored values in a new DataFrame column, **zRT**.



Hint: Remember `DataFrame.groupby` and `DataFrame.apply`.

## Section 2: Visualization

a) Using Seaborn, visualize the difference in (z-scored) reaction times as a function of interference. Try out different plotting styles and see what you like best.

b) Using Seaborn, visualize the difference in (z-scored) reaction times as a function of DBS. Try out different plotting styles and see what you like best. Which contrast (interferece vs. DBS) seems to have the larger effect?

c) As an optional challenge, try to plot both contrasts (ON vs. OFF, Control vs. Intereference) simultaneously. 

Hint: Read up on the `hue` argument in Seaborn.

## Section 3: Statistics

a) Use `DataFrame.groupby` to calculate the average z-scored response time difference (collapsing across patients) for both contrasts (i.e. Inteference - Control, DBS ON - DBS OFF).

b) Perform an independent samples t-test to test for significance differences in the two contrasts. Which effect is larger?